Example II.1.2

SINGLE FACTOR MODEL
Single Index Model

OLS Estimates Of Alpha and Beta For a Portfolio of Two Stocks (60% invested in AMEX and 40% invested in CSCO)

In [1046]:
#Import statements
import os
import pandas as pd
import numpy as np
import statsmodels.api as sm
import math

In [1047]:
#Arrays for storing input data
arr_stock_price_data = []
arr_portfolio_allocation_data = []

#Arrays for storing calculated data
arr_daily_stock_lognormal_returns = []

Determine location of script

In [1048]:
script_path = os.path.abspath('__file__') # i.e. /Users/yashwantbhambhani/Documents/SHWAYB/ipython/CarolPythonCode/2.1/2.1.1/__file__
script_dir = os.path.split(script_path)[0] #i.e. /Users/yashwantbhambhani/Documents/SHWAYB/ipython/CarolPythonCode/2.1/2.1.1

Load data from the input file

In [1049]:
arr_stock_price_csv_data = np.genfromtxt (script_dir + '/data/EX_II.1.2/Stocks_Prices.csv', delimiter=",")
#dtype = None has been specified in the next line of code to account for the first column being a string
arr_portfolio_allocation_csv_data = np.genfromtxt(script_dir + '/data/EX_II.1.2/Portfolio_Allocation.csv',delimiter=",",dtype=None)

Calculate Daily Log Normal returns

In [1050]:
#Remove the 'Date' column and first row corresponding to column headers containing stock symbols
#stock_price_data in the line below also includes data for S&P 500 index
arr_stock_price_data = arr_stock_price_csv_data[1:,1:]

for i in xrange(1, len(stock_price_data)):
        arr_daily_stock_lognormal_returns.append(np.log(arr_stock_price_data[i] / arr_stock_price_data[i-1]))

#Create dataframe from array of lognormal returns
df_daily_stock_lognormal_returns = pd.DataFrame(arr_daily_stock_lognormal_returns)
#Specify column names to each column of the DataFrame consisting of lognormal returns
df_daily_stock_lognormal_returns.columns = ['SP500', 'AMEX', 'CSCO']

Calculate portfolio returns

In [1051]:
#Transpose the data in portfolio_allocation_csv_data to make it easier to get weights of stocks in the portfolio
df_portfolio_allocation = pd.DataFrame(arr_portfolio_allocation_csv_data).T
df_portfolio_allocation.columns = ['SP500', 'AMEX', 'CSCO']

df_daily_SP500_rtn = df_daily_stock_lognormal_returns['SP500']
df_daily_portfolio_AMEX_rtn = df_daily_stock_lognormal_returns['AMEX'] * df_portfolio_allocation['AMEX'].get_value(1, 0)
df_daily_portfolio_CSCO_rtn = df_daily_stock_lognormal_returns['CSCO'] * df_portfolio_allocation['CSCO'].get_value(1, 0)
df_daily_portfolio_rtn = df_daily_portfolio_AMEX_rtn + df_daily_portfolio_CSCO_rtn

# OLS Regression Output for AMEX returns versus SP500 returns 

In [1052]:
y = df_daily_stock_lognormal_returns['AMEX']  # response
X = df_daily_SP500_rtn  # predictor
X = sm.add_constant(X)  # Adds a constant term to the predictor
model = sm.OLS(y, X)
ols_amex_sp500_results = model.fit()
print(ols_amex_sp500_results.summary())

                            OLS Regression Results                            
Dep. Variable:                   AMEX   R-squared:                       0.508
Model:                            OLS   Adj. R-squared:                  0.508
Method:                 Least Squares   F-statistic:                     2071.
Date:                Wed, 03 Aug 2016   Prob (F-statistic):          2.86e-311
Time:                        20:43:55   Log-Likelihood:                 5702.9
No. Observations:                2009   AIC:                        -1.140e+04
Df Residuals:                    2007   BIC:                        -1.139e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          0.0002      0.000      0.554      0.5

# OLS Regression Output for CSCO returns versus SP500 returns

In [1053]:
y = df_daily_stock_lognormal_returns['CSCO']  # response
X = df_daily_SP500_rtn  # predictor
X = sm.add_constant(X)  # Adds a constant term to the predictor
model = sm.OLS(y, X)
ols_csco_sp500_results = model.fit()
print(ols_csco_sp500_results.summary())

                            OLS Regression Results                            
Dep. Variable:                   CSCO   R-squared:                       0.433
Model:                            OLS   Adj. R-squared:                  0.433
Method:                 Least Squares   F-statistic:                     1534.
Date:                Wed, 03 Aug 2016   Prob (F-statistic):          8.92e-250
Time:                        20:43:55   Log-Likelihood:                 4696.4
No. Observations:                2009   AIC:                            -9389.
Df Residuals:                    2007   BIC:                            -9378.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const         -0.0002      0.001     -0.424      0.6

# Alternative 1: Determining Portfolio Beta using
OLS Regression Output for Portfolio returns versus SP500 returns

In [1054]:
y = df_daily_portfolio_rtn  # response
X = df_daily_SP500_rtn  # predictor
X = sm.add_constant(X)  # Adds a constant term to the predictor
model = sm.OLS(y, X)
ols_portfolio_sp500_results = model.fit()
print(ols_portfolio_sp500_results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.681
Model:                            OLS   Adj. R-squared:                  0.681
Method:                 Least Squares   F-statistic:                     4284.
Date:                Wed, 03 Aug 2016   Prob (F-statistic):               0.00
Time:                        20:43:55   Log-Likelihood:                 6120.7
No. Observations:                2009   AIC:                        -1.224e+04
Df Residuals:                    2007   BIC:                        -1.223e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const       1.665e-05      0.000      0.065      0.9

# Alternative 2: Calculating Portfolio Alpha and Beta using 
1. Alpha and Beta of AMEX and CSCO and 
2. Weights of the two equity positions in the equity portfolio

In [1055]:
portfolio_alpha = df_portfolio_allocation['AMEX'].get_value(1, 0) * ols_amex_sp500_results.params["const"] + df_portfolio_allocation['CSCO'].get_value(1, 0) * ols_csco_sp500_results.params['const']
portfolio_beta = df_portfolio_allocation['AMEX'].get_value(1, 0) * ols_amex_sp500_results.params["SP500"] + df_portfolio_allocation['CSCO'].get_value(1, 0) * ols_csco_sp500_results.params["SP500"]
print "Portfolio Alpha: ", portfolio_alpha
print "Portfolio Beta: ", portfolio_beta
print "Standard Error of Coefficient: ", ols_amex_sp500_results.bse
print "Standard Error of Regression: ", math.sqrt(ols_amex_sp500_results.mse_resid)
#print dir(ols_amex_sp500_results)
#     for attr in dir(ols_amex_sp500_results):
#             print "ols_amex_sp500_results.%s = %s" % (attr, getattr(ols_amex_sp500_results, attr))

Portfolio Alpha:  1.66515397942e-05
Portfolio Beta:  1.44862663412
Standard Error of Coefficient:  const    0.000316
SP500    0.027250
dtype: float64
Standard Error of Regression:  0.0141633068665


# SUMMARY #1

In [1056]:
arr_alpha = [ols_amex_sp500_results.params["const"], ols_csco_sp500_results.params["const"], ols_portfolio_sp500_results.params["const"]]
arr_beta = [ols_amex_sp500_results.params["SP500"], ols_csco_sp500_results.params["SP500"], ols_portfolio_sp500_results.params["SP500"]]
arr_std_error = [math.sqrt(ols_amex_sp500_results.mse_resid), math.sqrt(ols_csco_sp500_results.mse_resid), math.sqrt(ols_portfolio_sp500_results.mse_resid)]
arr_specific_risk = [math.sqrt(ols_amex_sp500_results.mse_resid)*math.sqrt(250),
                     math.sqrt(ols_csco_sp500_results.mse_resid)*math.sqrt(250),
                     math.sqrt(ols_portfolio_sp500_results.mse_resid)*math.sqrt(250)]

df_summary_temp = pd.DataFrame({'Alpha':arr_alpha,'Beta':arr_beta,'Standard Error':arr_std_error,'Specific Risk':arr_specific_risk})
column_order = ['Alpha', 'Beta', 'Standard Error', 'Specific Risk']
df_summary_temp = df_summary_temp[column_order]
#Transpose the df_summary_temp dataframe 
df_summary = df_summary_temp.T
df_summary.columns = ['AMEX', 'CSCO', 'Portfolio']
print(df_summary)

                    AMEX      CSCO  Portfolio
Alpha           0.000175 -0.000221   0.000017
Beta            1.240011  1.761550   1.448627
Standard Error  0.014163  0.023374   0.011504
Specific Risk   0.223942  0.369573   0.181891


Specific Risk in the above summary is based on regressing the constant weighted return series {0.6 * AMEX Return + 0.4 * CSCO Return} on the index returns

# Specific Risk (Method 2) of portfolio using weights of stocks in the portfolio and specific risk of individual stocks

In [1057]:
specific_risk_2 = math.sqrt(math.pow(df_summary['AMEX']['Specific Risk']*df_portfolio_allocation['AMEX'].get_value(1, 0),2)+
                        math.pow(df_summary['CSCO']['Specific Risk']*df_portfolio_allocation['CSCO'].get_value(1, 0),2))


# Systematic Risk Calculation using Beta of the portfolio and standard deviation of the SP100 index

In [1058]:
sp500_vol = np.std(df_daily_SP500_rtn)*math.sqrt(250)
ols_systematic_risk = ols_portfolio_sp500_results.params["SP500"] * sp500_vol

# Correlation between AMEX / SP500 daily lognormal returns & CSCO / SP500 lognormal returns

In [1059]:
amex_sp100_corr = np.corrcoef(df_daily_stock_lognormal_returns['SP500'], df_daily_stock_lognormal_returns['AMEX'])
csco_sp100_corr = np.corrcoef(df_daily_stock_lognormal_returns['SP500'], df_daily_stock_lognormal_returns['CSCO'])

# Summary #2

In [1060]:
print "Specific Risk of Portfolio (Method 2)", specific_risk_2
print "SP100 Volatility: ", sp500_vol
print "OLS Systematic Risk (Method 2): ", ols_systematic_risk
print "AMEX_SP100 Correlation: ", amex_sp100_corr[0,1]
print "CSCO_SP100 Correlation: ", csco_sp100_corr[0,1]

Specific Risk of Portfolio (Method 2) 0.199768307624
SP100 Volatility:  0.183346945592
OLS Systematic Risk (Method 2):  0.265601268669
AMEX_SP100 Correlation:  0.71260517898
CSCO_SP100 Correlation:  0.658227114926


# Conclusion - Which Specic Risk of Portfolio should be used?

The one computed by the output of regression between constant weighted return series on the index returns